In [ ]:
import cv2
import numpy as np 
import pandas as pd
import glob
import csv
import os
import pdb

In [ ]:
#original_image_folder = './test/original/' #path to original image folder
gt_image_folder = '' #path to gt image folder
#original_images = glob.glob(original_image_folder + '*.jpg') + glob.glob(original_image_folder + '*.png') +  glob.glob(original_image_folder + '*.jpeg')
gt_images = glob.glob(gt_image_folder + '*.jpg') + glob.glob(gt_image_folder + '*.png') +  glob.glob(gt_image_folder + '*.jpeg') + glob.glob(gt_image_folder + '*.JPEG')
label_name = '' #set name of the label

## Create Bounding Box and Write CSV File:

In [ ]:
os.makedirs('bbox',exist_ok=True)
training_path = '' #Training data file path
new_dict = {'wd':[],'bbox':[]}

for j,i in enumerate(gt_images):
        img = cv2.imread(i)
        ret,thresh = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),127,255,cv2.THRESH_BINARY)
        image,contours,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        bbox_list = []
        x = None
        for c in contours:
            x,y,w,h = cv2.boundingRect(c)
            #pdb.set_trace()
            #only keep large boxes or 0 0 0 0 (negative example) boxes
            
            if w>1 and w>1 : #suppress small bounding boxes (optional)
                #bbox_list.extend([x,y,x+w-1,y+h-1])
                bbox_list.extend([x,y,w,h])
                cv2.rectangle(img,(x,y),(x+w-1,y+h-1),(0,255,0),1)
            else: 
                continue
        #new_dict.setdefault('fn', []).append(i.replace('.png','.jpeg').replace('\\','/').replace(gt_image_folder,training_path))
        if bbox_list: #remove gt files that do not contain any bboxes
            new_dict.setdefault('wd', []).append(i.replace('\\','/').replace(gt_image_folder,training_path).split('_')[0]+'.png')
            new_dict.setdefault('bbox', []).append(' '.join (str(p) for p in bbox_list))
            #draw bounding boxes on gt images
            cv2.imwrite('bbox_MATest/{}_BBox.png'.format(i.split('/')[1].split('.')[0]),img)
            #print (x,y,w,h)
            print('writing bbox_MATest/{}_BBox.png...'.format(j))
        else:
            #output 0,0,0,0 for empty image (negative training samples)
            bbox_list.extend([0,0,1,1])
            new_dict.setdefault('wd', []).append(i.replace('\\','/').replace(gt_image_folder,training_path).split('_')[0]+'.png')
            new_dict.setdefault('bbox', []).append(' '.join (str(p) for p in bbox_list))

In [ ]:
df = pd.DataFrame(new_dict, columns=['wd','bbox'])
df.to_csv('.csv', index=False) #change the name of the output .csv file

In [ ]:
print(bbox_list)

## End